In [3]:
import re
import os
import numpy as np
import glob
from colorama import Fore, Back, Style
from tqdm import tqdm
import pandas as pd
from subprocess import Popen, PIPE, call, check_call
import datetime
import shutil
from datetime import datetime
import time

from help_functions import replace_gpr #user def
from lut import tests #user def 


## General defines, paths etc

In [26]:
transcript_path = os.environ['NOELV']
grlib_path = os.environ['GRLIB']

riscv_dv = "/home/jonathanjonsson/MasterThesis/riscv-dv/"
test_number = 3
print("Running the test: {}".format(tests[test_number]))
now = datetime.now()
current_time = now.strftime("%Y-%m-%d")

Running the test: riscv_loop_test


## Generate RISCV-DV tests and run OVPsim
- Tests that work: 0
- Tests that sometimes timeout but sometimes doesn't: 1, 2, 10
- Tests that fail due to add instruction after **mret**: 3, 4
- Tests that doesn't run at all: 5, 6 (Never enters program after running the bootloader)
- Test 7 seems to outright fail
- Test 8,9 is maybe not applicable since ebreak is handled differently between ovpSIM and NOEL-V

In [ ]:
param = "python3 run.py --custom_target target/rv64_noelv/ --iss ovpsim --simulator riviera --isa rv64gc --mabi lp64 --test "+tests[test_number]+" --iterations 1"
process = Popen('cd '+riscv_dv+' && source ~/.bashrc && module load aldec/riviera/2021.10 &&'+param, shell=True, stdout=PIPE, stderr=PIPE)
#process.wait()
stdout, stderr = process.communicate()
timeout = False
error = stderr.decode('ascii')

if(error.find("You do not have a valid license to run Riviera-PRO") == -1):
    if(error.find("Timeout") == -1):
        print("No timeout")
        timeout = False
    else:
        print("OVPsim timed out")
        timeout = True
else:
    raise RuntimeError("License not found for Riviera PRO")

no_instr_timeout = 1000000
clock_period = 10 #ns
ipc_compensate = 1.2 #compensate with 20% extra simulation time in case ipc != 1

no_timeout_arg = ['run -all\n', 'quit\n', '\n']
timeout_arg = ['run '+str(int(no_instr_timeout*clock_period*ipc_compensate))+'ns\n', 'quit\n', '\n']

with open(grlib_path+"/bin/runvsim.do","w") as f:
    if(timeout):
        f.writelines(timeout_arg)
    else:
        f.writelines(no_timeout_arg)
f.close()
    

## Run RTL simulation
### For now only consider tests generated today
- How do we break simulation if we have a timeout in OVPsim?
    - It seems like execution time scales quite well to number of instructions*clock_period, add 20% extra sim time to compensate for IPC differences

In [ ]:
srec_tests = glob.glob(riscv_dv+"out_"+current_time+"/asm_test/"+tests[test_number]+"*.srec")
shutil.copy(srec_tests[0], transcript_path+'ram.srec') #copy the current test to the sim dir
print("Copied {} to the NOELV directory".format(srec_tests))

#launch sim
process = Popen('cd '+transcript_path+' && source ~/.bashrc && module add mentor/questasim/2021.3 && make sim-run', shell=True, stdout=PIPE, stderr=PIPE)
#process.wait()
stdout, stderr = process.communicate()
if(stdout.decode('ascii').find("# ** Failure: *** IU in error mode, simulation halted ***") == -1):
    print("RTL Simulation failed")

if not os.path.exists(riscv_dv+"out_"+current_time+"/rtl_log/"):
    os.mkdir(riscv_dv+"out_"+current_time+"/rtl_log/")
shutil.copy(transcript_path+"transcript", riscv_dv+"out_"+current_time+"/rtl_log/"+tests[test_number]+"_"+re.search("\d*(?=.srec)",str(srec_tests)).group(0)+".log")

## Parse RTL transcript and create rtl_log

In [21]:
lookup = re.compile("#\s*\d*\s*ns\s*:\sC\d I\d : \d*\s*\[\d\] @0x[0-9a-fA-F]{16} \(0x[0-9a-fA-F]{4,8}\)\s*.*")
match = 0
f = open(transcript_path+"transcript")
transcript = f.readlines()
f.close()

#regex to extract content relevant to rtl sim
error_re = re.compile("[eE]rror | [wW]arning | [fF]ailure")
pc_csr_status_tval_re = re.compile("[0-9a-fA-F]{16}|X{16}")
binary_re = re.compile("(?<=(\(0x))[a0-z9]{4,8}(?=\))")
instruction_re = re.compile("\w{2,8}(?=(\s\w{2,3},))|(\w*\.[a-z])|nop|mret|ecall|fence|unknown instruction|ebreak")
instr_str_re = re.compile("\w*\s\w*,\s\w*,\s-?\w*(?=\s*W)|\w*(?=\s*W)|\w*\s\w*,\s\w*(?=\s*W)") #ugly
gpr_re = re.compile("(?<=([a-z]{2}\s))\w{2,3}(?=,)")
op1_re = re.compile("(?<=,\s)\w{1,8}((?=,)|(?=\s*W))|0\(\w{2,3}\)") 
op2_re = re.compile("(\w*,\s\w*,\s-?\w*)") #use split to get op2, don't have a nicer regex atm
csr_re = re.compile("(?<=\[)\w{2,3}(?=\s?=)")
mode_re = re.compile("(?<=PRV\[)\d*")
valid_re = re.compile("(?<=\[)\d(?=\]\s@)")
exception_re = re.compile("(?<=\[)\d(?=\]\sPRV)")

pc = []
binary = []
instr = []
instr_str = []
gpr = []
csr = []
gpr_val = []
op_0 = []
op_1 = []
status = []
tval = []
mode = []
exception = []


for ln,line in tqdm(enumerate(transcript)):
#    print(line)
    if(re.search(lookup,line) and re.search(valid_re, line)): #check that line conforms to format then extract information
        pc_i = re.search(pc_csr_status_tval_re, line)
        if(pc_i):
            pc.append(pc_i.group(0))
        else:
            print(line)
            break
        binary.append(re.search(binary_re, line).group(0))
        instruction = re.search(instruction_re, line)
        if(instruction):
            instr.append(instruction.group(0))
        else:
            raise RuntimeError("Instruction not present in RegEx\n"+line)
        instr_str.append(re.search(instr_str_re, line).group(0))
        match3 = re.search(gpr_re, line)
        if(match3):
            gpr.append(match3.group(0))
        else:
            gpr.append("")
        match4 = re.search(op1_re, line)
        if(match4):
            op_0.append(match4.group(0))
        else:
            op_0.append("")
        match5 = re.search(op2_re,line)
        if(match5):
            op_1.append(match5.group(0).split(", ")[2])
        else:
            op_1.append("")
        match6 = re.search(csr_re,line)
        if(match6):
            csr.append(match6.group(0))
        else:
            print(line)
            raise RuntimeError("CSR RegEx not complete")

        match7 = re.findall(pc_csr_status_tval_re,line)
        if(len(match7) == 3):
            print(match7)
            print(line)
        gpr_val.append(match7[1])
        status.append(match7[2])
        tval.append(match7[3])
        mode.append(re.search(mode_re,line).group(0))
        exception.append(re.search(exception_re,line).group(0))



columns = ['pc','instr','gpr','csr','binary','mode','instr_str','operand','exception']
rtl_log = pd.DataFrame(columns=columns)
rtl_log[columns[0]] = pc
rtl_log[columns[1]] = instr
rtl_log[columns[2]] = pd.Series(gpr) +":"+ pd.Series(gpr_val)
rtl_log[columns[3]] = csr
rtl_log[columns[4]] = binary
rtl_log[columns[5]] = mode
rtl_log[columns[6]] = instr_str
rtl_log[columns[7]] = pd.Series(gpr)+","+pd.Series(op_0)+","+pd.Series(op_1)
rtl_log[columns[8]] = exception
#log[columns[9]] = tval
rtl_log = rtl_log[41:].reset_index(drop=True) #41 is where the bootloader ends
def replace_gpr(pd_series,gpr,re_gpr):
    tmp = []
    for line in tqdm(pd_series):
        split_line = line.split(":")
        if(len(split_line[0]) > 0 and split_line[0] == gpr):
            split_line[0] = re_gpr
        ln = split_line[0]+":"+split_line[1]
        tmp.append(ln)
    return pd.Series(tmp)
rtl_log.gpr = replace_gpr(rtl_log.gpr, 'fp','s0') #replace fp with s0 to keep convention consistent
#rtl_log = rtl_log[~((rtl_log['instr'] == "addi") & (rtl_log.shift(1)['instr'] == "mret"))].reset_index(drop=True) #deal with gaislers addi injection after mret, this removes all of those lines


15892it [00:01, 10712.58it/s]
100%|██████████| 12295/12295 [00:00<00:00, 889656.99it/s]


## Convert OVPsimlog to csv

In [11]:
logs = glob.glob(riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.log")
csvs = glob.glob(riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.csv")
for log in logs:
    process = Popen("python3 "+riscv_dv+"scripts/ovpsim_log_to_trace_csv.py --dont_truncate_after_first_ecall --log "+log+ " --csv "+log.strip(".log")+".csv", shell=True, stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
csvs = glob.glob(riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.csv")
sim_log = pd.read_csv(csvs[0])
sim_log['mode'] = sim_log['mode'].astype('Int64')
sim_log['gpr'] = sim_log['gpr'].astype('str')
if(not timeout):
    sim_log = sim_log.truncate(after=len(sim_log)-3) #exit sequence slightly different between the rtl_sim and ovpsim

## Compare series to see if run was successfull

In [13]:
if(abs(len(sim_log)-len(rtl_log) < 1000)):
#if(abs(len(sim_log)-len(rtl_log) < 1000000)):

# This limit is somewhat arbitrary, not sure what best course of action is for different length logs.
# We can expect some difference in length when OVPsim timeouts since the runtime is then arbitrarily set, however, the rtl_log should be almost the same length
# as that of the sim_log. 
    
    if(len(sim_log) < len(rtl_log)):
        rtl_log=rtl_log.truncate(after=len(sim_log)-1)
        print("Truncating rtl_log to {} elements to meet size of sim_log".format(len(rtl_log)))
    elif(len(sim_log) == len(rtl_log)):
        print("Both logs are equally sized")

    else:
        sim_log=sim_log.truncate(after=len(rtl_log)-1)
        print("Truncating sim_log to {} elements to meet size of rtl_log".format(len(rtl_log)))
else:
    raise RuntimeError("Lengths of series differs with "+str(abs(len(sim_log)-len(rtl_log)))+" elements") 


matches = len(sim_log[(sim_log.gpr != "nan") & (rtl_log.gpr == sim_log.gpr)])
nan_count = len(sim_log[sim_log.gpr == "nan"])

tot_count = matches + nan_count
gpr_match = (tot_count == len(rtl_log.gpr))
binary_match = rtl_log.binary.eq(sim_log.binary).all()
pc_match = rtl_log.pc.eq(sim_log.pc).all()

if(gpr_match and binary_match and pc_match):
    print("GPR, Binary and PC contents match")
else:
    print("PC match: {}\nBinary match: {}\nGPR match: {} ".format(pc_match,binary_match,gpr_match))

Both logs are equally sized
GPR, Binary and PC contents match


In [14]:
rtl_log

,pc,instr,gpr,csr,binary,mode,instr_str,operand,pad,exception
0,0000000000000000,csrrs,t0:0000000000000000,t0,f14022f3,3,"csrrs t0, mhartid, x0","t0,mhartid,x0",[0],NaN
1,0000000000000004,addi,t1:0000000000000000,t1,4301,3,"addi t1, x0, 0","t1,x0,0",[0],NaN
2,0000000000000006,beq,t0:0000000000000000,tp,00628263,3,"beq t0, t1, 0x0000000000000000a","t0,t1,0x0000000000000000a",[0],NaN
3,000000000000000a,auipc,s1:000000000000000a,s1,00000497,3,"auipc s1, 0x00000","s1,0x00000,",[0],NaN
4,000000000000000e,addi,s1:0000000000000016,s1,00c48493,3,"addi s1, s1, 12","s1,s1,12",[0],NaN
...,...,...,...,...,...,...,...,...,...,...
12290,000000000000117a,sd,t5:0000000000000001,a6,0fe83823,3,,"t5,0(a6),",[0],NaN
12291,000000000000117e,sd,t6:00000000000005ce,s8,0ff83c23,3,,"t6,,",[0],NaN
12292,0000000000001182,auipc,s1:000000000000b182,s1,0000a497,3,"auipc s1, 0x0000a","s1,0x0000a,",[0],NaN
12293,0000000000001186,addi,s1:000000000000b4cc,s1,34a48493,3,"addi s1, s1, 842","s1,s1,842",[0],NaN
